# Modelling NIQS Hardware TF

In [1]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import torch.optim as optim
import tensorflow as tf

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from cost_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from src_torch import *


#from src_tf import *

In [13]:
def apply_map(state, kraus_list):
    state = [K@state@tf.linalg.adjoint(K) for K in kraus_list]
    state = tf.math.reduce_sum(tf.stack(state), axis=0)
    return state

def expectation_value(state, observable):
    ev = tf.linalg.trace(observable@state)
    return ev

In [14]:
random.seed(42)
np.random.seed(42)

n = 1
d = 2**n
rank = 2

G = tf.cast(tf.random.normal((rank*d, d), 0, 1), tf.complex64)
Q, R = tf.linalg.qr(G, full_matrices = False)
A = tf.linalg.tensor_diag_part(R)
A = tf.math.sign(A)
A = tf.linalg.diag(A)
U = Q@A


kraus_target_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]

In [15]:
N = 100
state_index, observ_index = index_generator(n, N, trace=False)

input_list = []
circuit_list = []
for i, j in zip(state_index, observ_index):

    config = numberToBase(i, 6, n)
    state = tf.convert_to_tensor(prepare_input(config), dtype = tf.complex64)

    config = numberToBase(j, 3, n)
    observable = tf.convert_to_tensor(pauli_observable(config),  dtype = tf.complex64)
    
    input_list.append([state, observable])

target_list = [expectation_value(apply_map(input[0], kraus_target_list), input[1]) for input in input_list]

G = tf.cast(tf.random.normal((rank*d, d), 0, 1), tf.complex64)

In [18]:
for i in range(100):
    with tf.GradientTape() as tape:

        tape.watch(G)
        Q, R = tf.linalg.qr(G, full_matrices = False)
        A = tf.linalg.tensor_diag_part(R)
        A = tf.math.sign(A)
        A = tf.linalg.diag(A)
        U = Q@A

        kraus_model_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]
        pred_list = [expectation_value(apply_map(input[0], kraus_model_list), input[1]) for input in input_list]
        loss = tf.math.reduce_mean(tf.stack([(target - predicted)**2 for target, predicted in zip(target_list, pred_list)]))

    grads = tape.gradient(loss, G)
    G = G - 0.05*grads
    print(f"loss: {loss: .4f}")

loss:  0.0096+0.0000j
loss:  0.0092+0.0000j
loss:  0.0089-0.0000j
loss:  0.0087-0.0000j
loss:  0.0084+0.0000j
loss:  0.0081-0.0000j
loss:  0.0078-0.0000j
loss:  0.0076+0.0000j
loss:  0.0073-0.0000j
loss:  0.0071+0.0000j
loss:  0.0069+0.0000j
loss:  0.0067+0.0000j
loss:  0.0064+0.0000j
loss:  0.0062+0.0000j
loss:  0.0060+0.0000j
loss:  0.0058-0.0000j
loss:  0.0056-0.0000j
loss:  0.0055-0.0000j
loss:  0.0053+0.0000j
loss:  0.0051+0.0000j
loss:  0.0049+0.0000j
loss:  0.0048-0.0000j
loss:  0.0046+0.0000j
loss:  0.0045+0.0000j
loss:  0.0043-0.0000j
loss:  0.0042-0.0000j
loss:  0.0040+0.0000j
loss:  0.0039+0.0000j
loss:  0.0038+0.0000j
loss:  0.0037+0.0000j
loss:  0.0035-0.0000j
loss:  0.0034+0.0000j
loss:  0.0033-0.0000j
loss:  0.0032+0.0000j
loss:  0.0031+0.0000j
loss:  0.0030+0.0000j
loss:  0.0029+0.0000j
loss:  0.0028+0.0000j
loss:  0.0027+0.0000j
loss:  0.0026+0.0000j
loss:  0.0025-0.0000j
loss:  0.0024+0.0000j
loss:  0.0024+0.0000j
loss:  0.0023+0.0000j
loss:  0.0022+0.0000j
loss:  0.0

## Test

In [ ]:
n = 2
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

np.random.seed(43)
tf.random.set_seed(43)

optimizer = tf.optimizers.SGD(learning_rate=1e-8)

X_target = generate_ginibre(d**2, 2)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]

X_model = generate_ginibre(d**2, 2)

for i in range(100):
    state_input = state_input_list[0]
    state_target = state_target_list[0]
    print("param", grads[0,0])
    
    with tf.GradientTape() as tape:
        tape.watch(X_model)
        choi_model = generate_choi(X_model)
        state_model = apply_map(state_input, choi_model)
        fid = -state_fidelity(state_model, state_target)
    
    grads = tape.gradient(fid, X_model)
    print(grads)
    #optimizer.apply_gradients(zip([grads], [X_model]))
    #print("fid", fid)
    